# CLUSTERS POLYGONS METHODS ENDPOINTS 

##### Obs: Obsolete code for api. This notebook's code is no longer used in api production.

In [1]:
cd ../../../../Apps/Python/bolsao-api

C:\Users\luisr\Desktop\Repositories\Apps\Python\bolsao-api


#### Import modules

In [4]:
# ---
# Import modules

from warnings import filterwarnings as fws; fws('ignore')
import json, pandas as pd
from copy import deepcopy

# ---
# Custom modules

from modules.polygons import Polygon
from modules.geojson_conversion import polygon_geojson
from modules.mongo_requests import last_prediction_record
from modules.comando import comando_open_events
from modules.waze import get_waze_partner_alerts

---
## Local resources

In [6]:
#### Polygons dataset

#### Load clusters
clusters_df = pd.read_csv('static/clusters/clusters_micro.csv')
clusters_df = clusters_df[clusters_df['sublabel'] != -1]

#### Polygons geojson
clusters_geojson = polygon_geojson(clusters_df)

#### Polygon class instance
polygons = Polygon(clusters_geojson, poly_id='sublabel')

---
## Columns' settings

In [7]:
key_id =' cluster_id'

#### Clusters geojson properties settings
geometry_cols = ['lng_min', 'lng_max', 'lat_min', 'lat_max']
keep_cols_clusters = [
    'sublabel', 'main_neighborhood', 'main_route', 'main_street_number_range',
    'lat_centroid', 'lng_centroid', 'label_count', 'area_box', 'area_circle'
]
main_cols_clusters = keep_cols_clusters + geometry_cols
waze_drop = []
comando_drop = []
prob_drop = ['_id', 'cluster', 'range']
drop_cols = [waze_drop, comando_drop, prob_drop] # cols to drop from requests - important

---
# Waze endpoints

#### Functions

In [11]:
def get_waze_alerts():
    _id, coords = 'uuid', ['latitude', 'longitude']
    waze_df = get_waze_partner_alerts()
    waze_df['cluster_id'] = polygons.points_belong(waze_df, coords, _id)
    return json.dumps(waze_df.to_dict(orient='records'))

def get_waze_alerts_geojson():
    _id, coords = 'uuid', ['latitude', 'longitude']
    waze_df = get_waze_partner_alerts()
    waze_df['cluster_id'] = polygons.points_belong(waze_df, coords, _id)
    return json.dumps(points_geojson(waze_df, coords=coords))

def get_waze_clusters():
    _id, coords = 'uuid', ['latitude', 'longitude']
    waze_df = get_waze_partner_alerts()
    polygons_waze = polygons.has_points(waze_df, _id, coords, key_id, prefix='waze_')
    return json.dumps(polygons_waze.to_dict(orient='records'))

### Examples

In [14]:
get_waze_partner_alerts().head()

,country,city,reportRating,confidence,reliability,type,uuid,roadType,magvar,subtype,street,pubMillis,nThumbsUp,reportDescription,latitude,longitude
0,BR,Rio de Janeiro,2,0,5,JAM,1115984a-624c-472b-944e-61619efbe4d5,6.0,237,JAM_STAND_STILL_TRAFFIC,Av. Brasil - Pista Central,2022-12-23 11:50:49,,,-43.374517,-22.843305
1,BR,Rio de Janeiro,4,0,5,WEATHERHAZARD,d456e52a-a79b-4a95-bdf1-4fcdf4b09b42,6.0,68,HAZARD_ON_ROAD_CAR_STOPPED,Av. Brasil - Pista Central,2022-12-23 11:50:45,,,-43.353446,-22.832127
2,BR,Rio de Janeiro,2,0,5,JAM,c7f072e1-804c-47ce-b96e-89963865adec,3.0,345,JAM_HEAVY_TRAFFIC,Linha Vermelha,2022-12-23 11:50:44,,,-43.250823,-22.826621
3,BR,Rio de Janeiro,2,0,5,WEATHERHAZARD,ce68de89-0a8d-42b4-8a12-007f3c94a9c8,3.0,188,HAZARD_ON_ROAD_CAR_STOPPED,Linha Amarela,2022-12-23 11:50:44,,,-43.356873,-22.950927
4,BR,Rio de Janeiro,3,0,5,WEATHERHAZARD,0a41e964-c81f-4ab9-9196-189af33a0ac4,6.0,75,HAZARD_ON_ROAD_POT_HOLE,Av. Brasil - Pista Central,2022-12-23 11:50:33,,,-43.298550,-22.815072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,BR,Rio de Janeiro,0,5,10,ROAD_CLOSED,c322b21c-886c-492f-afb9-dd9f563475c0,1.0,0,ROAD_CLOSED_EVENT,Portaria 4,2022-06-24 18:00:41,0.0,,-43.232411,-22.861890
858,BR,Rio de Janeiro,0,5,10,ROAD_CLOSED,5e4db911-2227-44de-8b47-025dc4266aa3,2.0,0,ROAD_CLOSED_EVENT,,2022-06-17 17:00:50,0.0,,-43.207042,-22.910151
859,BR,Rio de Janeiro,0,5,10,ROAD_CLOSED,3c429ca3-cd66-4dcf-9f5e-5fe8a8ff4acb,2.0,0,ROAD_CLOSED_EVENT,,2022-05-14 11:08:23,0.0,,-43.364830,-22.837585
860,BR,Rio de Janeiro,0,5,10,ROAD_CLOSED,07342a6f-9d16-41ee-9f0e-ce485f3a776b,1.0,0,ROAD_CLOSED_EVENT,,2022-02-02 20:14:45,0.0,,-43.330305,-23.007484


---
# Comando endpoints

In [9]:
def get_comando_events():
    _id, coords = 'id', ['longitude', 'latitude']
    comando_df = comando_open_events()
    comando_df['cluster_id'] = polygons.points_belong(comando_df, coords, _id)
    return json.dumps(comando_df.to_dict(orient='records'))

def get_comando_events_geojson():
    _id, coords = 'id', ['latitude', 'longitude']
    comando_df = comando_open_events()
    comando_df['cluster_id'] = polygons.points_belong(comando_df, coords, _id)
    return json.dumps(points_geojson(comando_df, coords))

def get_comando_clusters():
    _id, coords = 'id', ['latitude', 'longitude']
    comando_df = comando_open_events()
    polygons_waze = polygons.has_points(comando_df, _id, coords, key_id, prefix='comando_')
    return json.dumps(comando_df.to_dict(orient='records'))

---
# Dinamic clusters endpoint functions

In [3]:
## Get and append current monitoring data to json
# def clusters_status(keep_geometry=False, drop_cols=drop_cols):
#     cols = keep_cols_clusters[:]
#     if keep_geometry: cols += geometry_cols
#     objs = deepcopy(clusters_df[cols]) # Copy features to keep originals intact
#     waze = get_clusters_waze_partner_alerts() # Waze data
#     comando = get_clusters_events() # Comando data
#     prob = pd.DataFrame(last_prediction_record(mongo_timeout=1000*5, load_limit=500)) # Probability data
#     for df, drop_cols in zip([waze, comando, prob], drop_cols):
#         objs = objs.join(df.set_index('cluster_id', drop=True).drop(drop_cols, axis=1), on='sublabel')
#     # Opertional logic
    
#     objs['status'] = status
    
#     return objs.fillna('')